## Tag 1: Muster und Algorithmus auswählen
Wählen Sie einen klassischen Bildverarbeitungs
Algorithmus zur Erkennung von Mustern wie Kanten, Linien, Kreisen oder Ecken.
Implementieren Sie jeden relevanten Schritt Ihres Mustererkennungsalgorithmus. Die
einzelnen Schritte des Algorithmus können Sie selbst programmieren oder Bibliotheken verwenden.
Wichtig ist jedoch, dass die jeweiligen Zwischenergebnisse der Schritte
in den Folgeaufgaben demonstriert werden können. Beginnen Sie ein neues Notebook.

(Ich nehme hier bereits, wie in Aufgabe 2 beschrieben ein eigenes Bild)


In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Bild laden (Graustufen)
img = cv2.imread("C:\\Users\\silas\\PycharmProjects\\GBSV_2025\\MC2\\Data\\Brienzersee.jpg", cv2.IMREAD_GRAYSCALE)

#Gauss-Filter zur Glättung (Rauschreduzierung)
blurred = cv2.GaussianBlur(img, (5, 5), 1.4)

# 2. Gradienten mit Sobel-Operator berechnen
sobel_x = cv2.Sobel(blurred, cv2.CV_64F, 1, 0, ksize=3)
sobel_y = cv2.Sobel(blurred, cv2.CV_64F, 0, 1, ksize=3)
gradient_magnitude = np.hypot(sobel_x, sobel_y)
gradient_direction = np.arctan2(sobel_y, sobel_x)

#Non-Maximum Suppression (vereinfacht)
def non_max_suppression(magnitude, direction):
    M, N = magnitude.shape
    output = np.zeros((M,N), dtype=np.float32)
    angle = direction * 180. / np.pi
    angle[angle < 0] += 180
    
    for i in range(1, M-1):
        for j in range(1, N-1):
            try:
                q = 255
                r = 255

                # Winkelbereiche einteilen
                if (0 <= angle[i,j] < 22.5) or (157.5 <= angle[i,j] <= 180):
                    q = magnitude[i, j+1]
                    r = magnitude[i, j-1]
                elif (22.5 <= angle[i,j] < 67.5):
                    q = magnitude[i+1, j-1]
                    r = magnitude[i-1, j+1]
                elif (67.5 <= angle[i,j] < 112.5):
                    q = magnitude[i+1, j]
                    r = magnitude[i-1, j]
                elif (112.5 <= angle[i,j] < 157.5):
                    q = magnitude[i-1, j-1]
                    r = magnitude[i+1, j+1]

                if (magnitude[i,j] >= q) and (magnitude[i,j] >= r):
                    output[i,j] = magnitude[i,j]
                else:
                    output[i,j] = 0

            except IndexError as e:
                pass

    return output

#Hysterese-Schwellenwertverfahren
def hysteresis(img, low, high):
    M, N = img.shape
    res = np.zeros((M,N), dtype=np.uint8)

    strong = 255
    weak = 75

    strong_i, strong_j = np.where(img >= high)
    weak_i, weak_j = np.where((img <= high) & (img >= low))

    res[strong_i, strong_j] = strong
    res[weak_i, weak_j] = weak

    # Verbindung schwacher mit starken Kanten (8-neighbors)
    for i in range(1, M-1):
        for j in range(1, N-1):
            if res[i,j] == weak:
                if ((res[i+1, j-1] == strong) or (res[i+1, j] == strong) or (res[i+1, j+1] == strong)
                    or (res[i, j-1] == strong) or (res[i, j+1] == strong)
                    or (res[i-1, j-1] == strong) or (res[i-1, j] == strong) or (res[i-1, j+1] == strong)):
                    res[i,j] = strong
                else:
                    res[i,j] = 0

    return res

